In [5]:
import torch

bs = 1
seq_len = 8192
d = 4096
D = 3 * d
mp_size = 1
num_groups = 256

groups_per_tp_rank = num_groups // mp_size
x = torch.randn(bs, D, seq_len)


In [6]:
def ref_interleave(x):
    x1 = x[:,0::3,:]
    x2 = x[:,1::3,:]
    v  = x[:,2::3,:]
    return x1, x2, v

In [7]:
x1_ref, x2_ref, v_ref = ref_interleave(x)

In [8]:
x1_ref.shape, x2_ref.shape, v_ref.shape

(torch.Size([1, 4096, 8192]),
 torch.Size([1, 4096, 8192]),
 torch.Size([1, 4096, 8192]))

In [9]:
x2 = x.reshape(bs, D // 3, 3, seq_len)
x3 = x2.permute(0, 2, 1, 3)
x3.shape

torch.Size([1, 3, 4096, 8192])

In [12]:
x4 = x3.unbind(dim=1)
x1, x2, v = x4
x1.shape, x2.shape, v.shape

(torch.Size([1, 4096, 8192]),
 torch.Size([1, 4096, 8192]),
 torch.Size([1, 4096, 8192]))

In [13]:
for ref, test in zip([x1_ref, x2_ref, v_ref], [x1, x2, v]):
    assert torch.allclose(ref, test)
